* pandas strinio sqlite tempfile
* Use http://www.ch-werner.de/sqliteodbc/ both 32 and 64 standard installs and then define it as an 
* ODBC 32-bit connection in adminstrator tools for use in Excel (and other apps)

In [1]:
import sqlite3
import pandas as pd
import io

table_name = "Planning"

Get data from https://kirkmcdonald.github.io/calc.html#items=lab:r:1 and load the data using the handy StringIO
https://docs.python.org/3/library/io.html?highlight=stringio#io.StringIO.  Using panda's read_csv function
https://stackoverflow.com/questions/41900593/csv-into-sqlite-table-python

In [2]:
scsv = """
item,item rate,factory,count,modules,beacon module,beacon count,power
logistic-science-pack,1,assembling-machine-1,0.2,,,,17000
inserter,1,assembling-machine-1,0.1,,,,3708.4
electronic-circuit,1,assembling-machine-1,0.1,,,,3708.4
copper-cable,3,assembling-machine-1,0.1,,,,4312.5
copper-plate,1.5,stone-furnace,0.1,,,,7200
copper-ore,1.5,electric-mining-drill,0.1,//,,,7350
transport-belt,1,assembling-machine-1,0.1,,,,3104.2
iron-gear-wheel,1.5,assembling-machine-1,0.1,,,,4312.5
iron-plate,5.5,stone-furnace,0.3,,,,26400
iron-ore,5.5,electric-mining-drill,0.2,//,,,18950
coal,0.504,electric-mining-drill,0.1,//,,,4461.6

"""
sio = io.StringIO(scsv)

df = pd.read_csv(sio)
df['Thing'] = df.item[0]
df[['item','item rate','factory','count','power','Thing']]

,item,item rate,factory,count,power,Thing
0,logistic-science-pack,1.000,assembling-machine-1,0.2,17000.0,logistic-science-pack
1,inserter,1.000,assembling-machine-1,0.1,3708.4,logistic-science-pack
2,electronic-circuit,1.000,assembling-machine-1,0.1,3708.4,logistic-science-pack
3,copper-cable,3.000,assembling-machine-1,0.1,4312.5,logistic-science-pack
4,copper-plate,1.500,stone-furnace,0.1,7200.0,logistic-science-pack
5,copper-ore,1.500,electric-mining-drill,0.1,7350.0,logistic-science-pack
6,transport-belt,1.000,assembling-machine-1,0.1,3104.2,logistic-science-pack
7,iron-gear-wheel,1.500,assembling-machine-1,0.1,4312.5,logistic-science-pack
8,iron-plate,5.500,stone-furnace,0.3,26400.0,logistic-science-pack
9,iron-ore,5.500,electric-mining-drill,0.2,18950.0,logistic-science-pack


Not forgetting to use pathlib to manage those Windows vs Linux path inconsistencies https://stackoverflow.com/questions/41900593/csv-into-sqlite-table-python

In [3]:
from pathlib import Path
filename = Path("factorio.db")

conn = sqlite3.connect(filename.absolute())
pd.io.sql.read_sql("SELECT item, \"item rate\", factory, count, power, Thing FROM Planning", conn).head(2)

,item,item rate,factory,count,power,Thing
0,logistic-science-pack,1.0,assembling-machine-1,0.2,17000.0,logistic-science-pack
1,inserter,1.0,assembling-machine-1,0.1,3708.4,logistic-science-pack


Create the table if it doesn't exist

In [4]:
conn.execute('CREATE TABLE IF NOT EXISTS ' + table_name +
                '("item"	TEXT,'
                '"item rate"	INTEGER,'
                '"factory"	TEXT,'
                '"count"	REAL,'
                '"modules"	TEXT,'
                '"beacon module"	REAL,'
                '"beacon count"	REAL,'
                '"power"	REAL,'
                '"Thing"	TEXT,'
                'PRIMARY KEY(item,Thing))')

append the data (the composite primary key ensures that duplicates will not be added).  Close the database connection irrespective of errors to avoid database corruption.

In [5]:
try:
  df.to_sql(name=table_name, con=conn, if_exists='append', index=False)
except:
  print("Something went wrong, probably duplicate data")
finally:
  conn.close()

Something went wrong, probably duplicate data
C:\Users\jackc\.conda\envs\Pandas\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Data can be viewed using https://sqlitebrowser.org/dl/ and used in Excel using this free SQLite ODBC driver http://www.ch-werner.de/sqliteodbc/.

Both the 32 and 64 standard installs need to be run befoe defining the ODBC connection using Windows within Control Panel -> Administrative Tools -> ODBC Data Sources (32-bit).  This assumes that 32-bit Excel is being used.

Could use SQLAlchemey for connection as it's faster than the standard one.  Manages the connection state too: closes them.

In [10]:
from sqlalchemy import create_engine
cnx = create_engine("sqlite:///" + filename.absolute().as_uri() + "?mode=ro&uri=true")
input_length = len(df.index)
stored_data = pd.io.sql.read_sql("SELECT item, \"item rate\", factory, count, power, Thing FROM Planning", cnx).tail(input_length)
input_data = df[['item','item rate','factory','count','power','Thing']]
if input_data.equals(stored_data):
    print("The stuff written matches the stuff input")
else:
    print("there's a problem: what is input doesn't match what was written")

The stuff written matches the stuff input


Previously_Using_TemporaryFile


In [13]:
import tempfile
fp = tempfile.TemporaryFile() 
b = bytes(scsv,'utf-8' )
fp.write(b)
fp.seek(0)
print("Stuff written to a temporary file: temporary file now in bit bucket.")
# ... stuff ...
fp.close()

Stuff written to a temporary file: temporary file now in bit bucket.
